# import lib

In [2]:
import json
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import MinMaxScaler

## preprprocess_string

remove vietnamese accents

https://gist.github.com/J2TEAM/9992744f15187ba51d46aecab21fd469

remove special character, only keep word and number

In [3]:
def preprprocess_string(a):
    a = a.lower().replace("\n"," ")
    
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    
    for c in a:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    
    s = re.sub('[^A-Z a-z0-9]+', '', s)
    return s

preprprocess_string("Pin sạc dự phòng Zendure 10,000 mAh 18W C3 ZDJP221B - Hàng Chính Hãng.")

'pin sac du phong zendure 10000 mah 18w c3 zdjp221b  hang chinh hang'

## processing shop data

create Dataframe from json file

category section is an array containing the category_id

This array has length from 2 to 4 so will add zeros to add nan data

In [4]:
def item_json_to_object(json_file_path):
    shopid = json_file_path.split("_")[0]
    itemid = json_file_path.split("_")[1].split(".")[0]

    item_details = open("item_data/"+json_file_path,"r")
    item_details = json.load(item_details)
    item_name = preprprocess_string( item_details["name"])
    item_price = item_details["price"]
    item_rating = item_details["item_rating"]["rating_star"]
    item_sold = item_details["historical_sold"]
    item_category = []

    for i in item_details["categories"]:
        item_category.append(int(i["catid"]))
    # categories is an array of int > 0
    # len of this array is the number from (2-4)
    # so i adding 0 to the null value
    if len(item_category) == 2:
        item_category.append(0)
        item_category.append(0)
    if len(item_category) == 3:
        item_category.append(0)
        
    item_ctime = item_details["ctime"]
    item_brand_id = item_details["brand_id"]
    item_comment_count = item_details["cmt_count"]
        
    return({
        "item_id": int(itemid),
        "shopid": int(shopid),
        "item_name":item_name,
        "item_price":int(item_price/10000000),
        "item_rating":item_rating,
        "item_sold":int(item_sold),
        "cat_0":item_category[0],
        "cat_1":item_category[1],
        "cat_2":item_category[2],
        "cat_3":item_category[3],
        "item_ctime":int(item_ctime),
        "item_brand_id":int(item_brand_id),
        "item_comment_count":int(item_comment_count),
    })
    
list_item = os.listdir("./data/item_data/")

item_data = []
for i in list_item: 
    item_data.append(item_json_to_object(i))

len(item_data)
item_data = pd.DataFrame(item_data)
item_data

FileNotFoundError: [Errno 2] No such file or directory: 'item_data/'

Check nan values in dataframe

In [ ]:
item_data.isnull().values.any()

False

In [ ]:
item_data.to_pickle("item_data.pkl")

# processing shop data

create Dataframe from json file

In [ ]:
def shop_json_to_object(json_file_path):
    shopid = json_file_path.split('.')[0]
    
    shop_details = open("shop_data/"+json_file_path,"r")
    shop_details = json.load(shop_details)
    
    shop_slug = shop_details["account"]["username"]
    rating_star = shop_details["rating_star"]
    response_rate = shop_details["response_rate"]
    response_time = shop_details["response_time"]
    follower_count = shop_details["follower_count"]
    rating_bad = shop_details["rating_bad"]
    rating_good = shop_details["rating_good"]
    rating_normal = shop_details["rating_normal"]
    item_count = shop_details["item_count"]
    
    return({
        "shopid" : int(shopid),
        "shop_slug" : shop_slug,
        "rating_star": rating_star,
        "response_rate": response_rate,
        "response_time": response_time,
        "follower_count" : follower_count,
        "rating_bad": rating_bad,
        "rating_good": rating_good,
        "rating_normal": rating_normal,
        "item_count": item_count
    })
    
list_shop = os.listdir("shop_data/")

shop_data = []
for i in list_shop: 
    shop_data.append(shop_json_to_object(i))

len(shop_data)
shop_data = pd.DataFrame(shop_data)
shop_data

,shopid,shop_slug,rating_star,response_rate,response_time,follower_count,rating_bad,rating_good,rating_normal,item_count
0,511068772,fpt_brand_store,4.578947,84.0,9733.0,3824,1,17,1,5
1,53587416,happystores.vn,4.893901,92.0,5009.0,2143,10,1175,18,194
2,284593651,philips_homeappliances,4.893010,100.0,2894.0,44977,104,9667,145,108
3,489924961,fpt_telecom_official,4.961240,99.0,4676.0,6828,0,128,2,8
4,239456796,supor_vn_official,4.880000,100.0,104.0,1718,1,145,6,34
...,...,...,...,...,...,...,...,...,...,...
85,258850486,amazfit_official_store,4.891045,98.0,7403.0,20377,16,2897,57,15
86,254666730,elmich_official_store,4.898768,80.0,9820.0,126813,52,6756,121,313
87,394941304,earldom.store,4.941433,98.0,3819.0,59430,50,16441,203,90
88,18645997,alobuy_official_store,4.803456,88.0,8043.0,8540,127,8073,295,766


check nan values in dataframe

In [ ]:
shop_data.isnull().values.any()

True

## Fill nan

can't replace nan with 0

Shops with nan data are usually new shops with insufficient number of purchases or reviews, 

so they will replace nan with the min value of column.

In [ ]:
# shop_data["rating_star"] = shop_data.fillna(shop_data.min(axis=1), inplace=True)
# shop_data
shop_data["rating_star"].fillna(shop_data['rating_star'].min(),inplace=True)
shop_data["response_rate"].fillna(shop_data['response_rate'].min(),inplace=True)
shop_data["response_time"].fillna(shop_data['response_time'].min(),inplace=True)
shop_data["follower_count"].fillna(shop_data['follower_count'].min(),inplace=True)
shop_data["rating_bad"].fillna(shop_data['rating_bad'].min(),inplace=True)
shop_data["rating_good"].fillna(shop_data['rating_good'].min(),inplace=True)
shop_data["rating_normal"].fillna(shop_data['rating_normal'].min(),inplace=True)
shop_data["item_count"].fillna(shop_data['item_count'].min(),inplace=True)
shop_data

,shopid,shop_slug,rating_star,response_rate,response_time,follower_count,rating_bad,rating_good,rating_normal,item_count
0,511068772,fpt_brand_store,4.578947,84.0,9733.0,3824,1,17,1,5
1,53587416,happystores.vn,4.893901,92.0,5009.0,2143,10,1175,18,194
2,284593651,philips_homeappliances,4.893010,100.0,2894.0,44977,104,9667,145,108
3,489924961,fpt_telecom_official,4.961240,99.0,4676.0,6828,0,128,2,8
4,239456796,supor_vn_official,4.880000,100.0,104.0,1718,1,145,6,34
...,...,...,...,...,...,...,...,...,...,...
85,258850486,amazfit_official_store,4.891045,98.0,7403.0,20377,16,2897,57,15
86,254666730,elmich_official_store,4.898768,80.0,9820.0,126813,52,6756,121,313
87,394941304,earldom.store,4.941433,98.0,3819.0,59430,50,16441,203,90
88,18645997,alobuy_official_store,4.803456,88.0,8043.0,8540,127,8073,295,766


check nan again

In [ ]:
shop_data.isnull().values.any()

False

## Calculate the score of each shop

Choose from the following fields ['rating_star','response_rate',
                                'response_time','follower_count',
                                'rating_bad','rating_good',
                                'rating_normal','item_count']
min max scaling 

### set weight for each data variable

If the data variable is more important, the absolute value of that variable will be larger

If the data variable has a negative meaning, then the variable will be negative

    [4],  # rating_star 

    [1],  # response_rate

    [-1],  # response_time  

    [3],  # follower_count

    [-3], # rating_bad

    [3],   # rating_good

    [1],   # rating_normal

    [0.5] # item_count

In [ ]:
shop_x = shop_data[['rating_star','response_rate',
                    'response_time','follower_count',
                    'rating_bad','rating_good',
                    'rating_normal','item_count']]
scaler = MinMaxScaler()
shop_x = scaler.fit_transform(shop_x)
shop_x.shape

NameError: name 'shop_data' is not defined

multiply all variable with weight

and minmax scaling again

In [ ]:
m1 = shop_x
m2 = np.array([
    [4],  # rating_star
    [1],  # response_rate
    [-1],  # response_time
    [3],  # follower_count
    [-3], # rating_bad
    [3],   # rating_good
    [1],   # rating_normal
    [0.5] # item_count
]
)
m3 = np.dot(m1,m2)
m3 = scaler.fit_transform(m3)
m3 = m3.reshape(m3.shape[0],)
m3

array([0.26872852, 0.50840162, 0.53817691, 0.55549249, 0.52102773,
       0.61290565, 0.48908217, 0.42357406, 0.28947519, 0.67088479,
       0.41883052, 0.5218126 , 0.43099839, 0.49084064, 0.54769132,
       0.49149987, 0.58019197, 0.47788492, 0.        , 0.66363235,
       0.5884746 , 0.49490438, 0.42938064, 0.46751852, 0.32834531,
       0.52975596, 0.29279036, 0.5458663 , 0.12753405, 0.46399764,
       0.49930374, 0.53580459, 0.4846679 , 0.43770108, 0.48201822,
       0.42549516, 0.45715509, 0.61705375, 0.47422509, 0.52519698,
       0.71913665, 0.41416813, 0.46327837, 0.48058289, 0.56258628,
       0.47190119, 0.37582631, 0.52062384, 0.48264292, 0.50565616,
       0.46219006, 0.57212403, 0.56210336, 0.39859608, 0.52271508,
       0.44940915, 0.53186092, 0.46171411, 0.55249269, 0.51117698,
       0.65885952, 0.50763218, 0.52495311, 0.51078488, 0.41174541,
       0.37958387, 0.46623013, 0.51102688, 0.35326286, 0.48477222,
       0.54428396, 0.47397851, 0.44249616, 0.48163356, 0.50877

In [ ]:
# shop_data["shop_score"] = m3
# shop_data = shop_data.sort_values(by="shop_score",ascending=False)
shop_score = shop_data[["shopid","shop_slug"]]
shop_score["shop_score"] = m3
shop_score = shop_score.sort_values(by="shop_score",ascending=False)
shop_score

<ipython-input-389-bb55788acd29>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shop_score["shop_score"] = m3


,shopid,shop_slug,shop_score
76,55771655,locknlockvn,1.000000
40,189615586,bluestone_vietnam,0.719137
9,65589552,samsung_official_store,0.670885
19,189431669,panasonic_official,0.663632
60,365903060,cowayvina_official_store,0.658860
...,...,...,...
26,441656386,dkb.official.store,0.292790
8,502901428,lazitech,0.289475
0,511068772,fpt_brand_store,0.268729
28,597699842,hosan_official,0.127534


now we notice the last row have score = 0

we don't want score = 0 because after that this score will multiply with the weight

so we need to change meet the condition 

average(row[-1] + row[-3]) == row[-2]

if row[-1] < 0 then row[-1] = row[-2] / 2

In [ ]:
# shop_data[-1]
a_1 = shop_score.iloc[-1]["shop_score"]
a_2 = shop_score.iloc[-2]["shop_score"]
a_3 = shop_score.iloc[-3]["shop_score"]

a_1_index = shop_data.iloc[-1]
a_1 = 2*a_2 - a_3
if a_1 <= 0:
    a_1 = a_2/2 
a_1

0.06376702426951766

In [ ]:
shop_score = shop_score.replace(0, a_1)
shop_score

,shopid,shop_slug,shop_score
76,55771655,locknlockvn,1.000000
40,189615586,bluestone_vietnam,0.719137
9,65589552,samsung_official_store,0.670885
19,189431669,panasonic_official,0.663632
60,365903060,cowayvina_official_store,0.658860
...,...,...,...
26,441656386,dkb.official.store,0.292790
8,502901428,lazitech,0.289475
0,511068772,fpt_brand_store,0.268729
28,597699842,hosan_official,0.127534


In [ ]:
shop_score.to_pickle("shop_data.pkl")